In [16]:
from bs4 import BeautifulSoup

with open('soco.txt', 'r', encoding='utf-8') as file:
    html_content = file.read()
soup = BeautifulSoup(html_content, 'html.parser')
elements_with_aria_labels = soup.find_all('div', attrs={
    "aria-label": True, 
    "role": "group"
})
all_labels = [el['aria-label'] for el in elements_with_aria_labels]

for label in all_labels:
    print(label)

1636 Leadership Forum
Active Minds Student Chapter at CAMHS
Adams House Committee
Advocating Success for Kids
All House Committee
An Evening with Champions
ApiChorus: Harvard Hillel Jewish Student A Capella Group
Armenian Student Association at Harvard College
Asian American Brotherhood
Asian Baptist Student Koinonia
Asian Student Arts Project
Association of Black Harvard Women
Athlete Ally: Harvard College Chapter
Australian Undergraduate Society at Harvard College ( AUS )
Bach Society Orchestra
Bengali Association of Students at Harvard College
Black Arts Collective
Black Community and Student Theater
Brattle Street Chamber Players
Cabot House Committee
Catholic Student Association
Center for European Studies Student Society
Center for Wellness and Health Promotions
Charles River Economics Labs
Class Committee
Club Sports Office
College Events Board
Colombian Students Association at Harvard College
Common Spaces
Concilio Latino de Harvard
Conflux
Contact Peer Counseling
Crimson Key S

In [17]:
import csv
mission_tags = soup.find_all('strong', string="Mission")
missions_data = []

for strong_tag in mission_tags:
    parent_p = strong_tag.find_parent('p')
    if parent_p:
        text = parent_p.get_text(separator=" ", strip=True)
        description = text.replace("Mission", "", 1).strip()
        missions_data.append({"Mission Description": description})

with open('missions_only.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=["Mission Description"])
    writer.writeheader()
    writer.writerows(missions_data)

In [34]:
import re
club_ids = [re.search(r'\d+', tag['id']).group() for tag in soup.find_all('input', id=re.compile(r'cb_club_\d+'))]

In [29]:
with open('id.csv', 'r') as f:
    club_ids = [row['ID'] for row in csv.DictReader(f)]
print(club_ids[:3])

['36314', '35446', '35450']


In [ ]:
import requests
import csv

# 1. Configuration
# REPLACE 'your_cookie_string_here' with the actual cookie from your browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Mobile Safari/537.36",
    "Cookie": "fpestid=5YkdV-DmfjHeZUf4p6sb_1k8LRezxsxsC58LHpGtehPpZd7yPOEeTlLsbRyn_K2N-S2mRw; kndctr_1B6E34B85282A0AC0A490D44_AdobeOrg_identity=CiYxNTI3NzY2ODQxMjI3ODM0NDk0MjY2OTE0NTg1MDc2OTY5NzM2MFIQCIyPhrykMxgBKgNWQTYwAfABjI-GvKQz; AMCV_1B6E34B85282A0AC0A490D44%40AdobeOrg=MCMID|15277668412278344942669145850769697360; __gads=ID=a83a3424e082fd3a:T=1762136262:RT=1762136262:S=ALNI_MZNv532W2BLaGb5ap_t71wBBa5Pmw; _hjSessionUser_5278425=eyJpZCI6IjdjMDZlYjdhLWY0YzEtNTlmYS1hNzFhLTkzYmMyM2ZmZGM3OSIsImNyZWF0ZWQiOjE3NjMwMDg4MDgyNTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjSessionUser_1721990=eyJpZCI6ImI5NDk3ODkzLTY1MWYtNTEwMy05N2NmLTNhYzkwNmMwM2Q1OSIsImNyZWF0ZWQiOjE3NjMwNDc5MTcyNjcsImV4aXN0aW5nIjpmYWxzZX0=; _hjSessionUser_5132978=eyJpZCI6Ijk3OTE2MzE3LWNkMzMtNThiYi05OGM3LWRlMWVkZTNiZmFjOCIsImNyZWF0ZWQiOjE3NjMxNzUyNDk4NjYsImV4aXN0aW5nIjpmYWxzZX0=; _hjSessionUser_3372373=eyJpZCI6ImI3NDA5NjRlLTAwOTktNTBkNi05MmIyLWQwZDhjNTY3NDhmNSIsImNyZWF0ZWQiOjE3NjQ3MjQwODE5OTYsImV4aXN0aW5nIjp0cnVlfQ==; cf_clearance=JM2tGC6MrVlp1kYjfJTe9vuioU099R7AhmOL.3_Yzow-1764894820-1.2.1.1-fJRUGfurPRRuT7om4PD_tbHOksMJ_g9pH4jck64DrR7BJvuni_9Nj2PWES8ff1Dm3tkBdmCMysFZ0eQgeyf4mVulmB6AP960Ihm7PrSXdGNalOZ7pVLMSL0IENHUN9JbPY9Ge2WZIZnZ98GexvSe.IHZHEGsUaQg6tMeBgmufYWEKQc43gGdiXDxt4Y9c86fA8ndQ107KaxK8qIGM0961C6SOjI3hAv0WWxxyS_KC3A; visid_incap_881684=/g7SFpDSTK+WiJmcA4KG2X0yMmkAAAAAQUIPAAAAAACFkJKV8lsbu/IKeRLk6lOK; _ga_MN44P5KS85=GS2.1.s1764895975$o6$g1$t1764897472$j60$l0$h0; _ga_HRDZW9PB73=GS2.2.s1764894720$o1$g1$t1764897501$j60$l0$h0; _ga_REG2399202=GS2.1.s1764897711$o2$g0$t1764897711$j60$l0$h0; _ga_72NC9RC7VN=GS2.1.s1764894542$o2$g1$t1764897739$j33$l0$h629955202; visid_incap_2811896=rOT45HlGToelhjz49dx6RHBVMmkAAAAAQUIPAAAAAABihYsb7FN2f7wFKiSfLhsX; _ga_08YNBBX8C4=GS2.1.s1764906776$o2$g1$t1764907572$j60$l0$h0; _ga_831NSELVRG=GS2.1.s1765224826$o2$g0$t1765224855$j31$l0$h0; _ga_XDRYG9V1BN=GS2.1.s1765426404$o4$g1$t1765426465$j60$l0$h0; _ga_6S7711N1RR=GS2.1.s1765426586$o1$g0$t1765426650$j60$l0$h0; _ga_F8NW5WEMSK=GS2.1.s1765462528$o3$g0$t1765462550$j38$l0$h0; _ga_1MSJRLKMVP=GS2.1.s1765462528$o12$g0$t1765462550$j38$l0$h0; _ga_3185NP2KZP=GS2.1.s1765462528$o3$g0$t1765462550$j38$l0$h0; _ga_GMEVN96PYN=GS2.1.s1765462551$o3$g0$t1765462678$j60$l0$h0; _ga_J818J4JRZP=GS2.2.s1765463024$o2$g0$t1765463024$j60$l0$h0; _ga_SFXEYPKNXD=GS2.1.s1765463277$o7$g0$t1765463315$j22$l0$h0; _ga_YQT8F7VJ08=GS2.1.s1765463277$o7$g0$t1765463315$j22$l0$h0; _ga_WKZX29Q985=GS2.1.s1765506975$o10$g1$t1765508970$j60$l0$h0; _ga_0BL4ZQBM0W=GS2.1.s1765547176$o13$g1$t1765548042$j56$l0$h0; _ga_W2SR8DW607=GS2.1.s1765547195$o2$g0$t1765548042$j60$l0$h0; _ga_2PZD5XNYSL=GS2.1.s1765547195$o2$g0$t1765548042$j60$l0$h0; _ga_2MCEMGXSJF=GS2.1.s1765548287$o1$g0$t1765548318$j29$l0$h0; _ga_2ESJCEX86M=GS2.1.s1765547182$o1$g1$t1765548394$j60$l0$h0; nmstat=4f450c70-67cd-5076-44f5-775f278b6af2; _ga_KBGFFL5T7T=GS2.1.s1765570918$o4$g1$t1765572278$j60$l0$h0; _ga_XPLDGPNPHJ=GS2.1.s1765628270$o17$g1$t1765628595$j47$l0$h0; _ga_ZH44FE6MF7=GS2.1.s1765628270$o17$g1$t1765628595$j47$l0$h0; _ga_Y1ZL3JYDGL=GS2.1.s1765673795$o1$g0$t1765673798$j57$l0$h0; _ga_HY4R7TDDW6=GS2.1.s1765673795$o1$g0$t1765673798$j57$l0$h0; _ga_3CXC97RWEK=GS2.1.s1765765432$o2$g1$t1765766047$j60$l0$h0; _ga_8VM586D95L=GS2.1.s1765768720$o14$g1$t1765768752$j28$l0$h0; _ga_WF7T5MH29J=GS2.1.s1766006435$o1$g1$t1766007547$j60$l0$h0; CG.SessionID=42gjdopagzgcmb2zcgi2o1gs-603MqGQjbijRfBxjphSu3myEPe4%3d; cg_uid=2404742-+hq5SB7vZ8wL8QVOleqUz9vb8qp9nT93X62aqSHSStc=; _ga_QEYJY07BXE=GS2.1.s1766374804$o14$g1$t1766375893$j10$l0$h0; _gid=GA1.2.1732466459.1766377655; _clck=1m005al%5E2%5Eg22%5E0%5E2125; _clsk=1lbycy8%5E1766377874682%5E9%5E1%5Ed.clarity.ms%2Fcollect; _ga_ECX5BHYZT7=GS2.1.s1766377654$o9$g1$t1766377876$j57$l0$h0; cebs=1; _ce.clock_data=33%2C100.38.155.175%2C1%2Cf18b5213b6de2490ec9be218b0f025b0%2CChrome%2CUS; cebsp_=1; _ce.s=v~aee173ba801a53c35bd281eddbc9c24aec63d1e1~lcw~1766379328747~vir~returning~lva~1766379165807~vpv~6~v11ls~0ecccc80-def2-11f0-ac40-bfc9e8dc6631~vdva~1765151999999~v11.cs~462316~v11.s~0ecccc80-def2-11f0-ac40-bfc9e8dc6631~v11.vs~aee173ba801a53c35bd281eddbc9c24aec63d1e1~v11.fsvd~eyJ1cmwiOiJzZWFzLmhhcnZhcmQuZWR1L25ld3MvKi8qL2dyZWVuLXNwYWNlcy1yZWQtbGluZXMtY3JpbXNvbi1zY2llbnRpc3RzIiwicmVmIjoiaHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8iLCJ1dG0iOltdfQ%3D%3D~v11.sla~1766379166025~v11.wss~1766379166026~v11.ss~1766379166027~gtrk.la~mjgon97s~lcw~1766379329128; _ga_Q97GE6JKJ9=GS2.1.s1766379165$o16$g1$t1766379333$j60$l0$h0; _ga_FNNHYP076Y=GS2.1.s1766379165$o15$g1$t1766379333$j12$l0$h0; _ga=GA1.2.566895420.1765599137; _ga_ECPW5BRBQ7=GS2.2.s1766409177$o1$g1$t1766409177$j60$l0$h0",
    "Accept": "application/json"
}
output_file = 'harvard_clubs_data_officer.csv'

def get_club_data(club_id):
    url = f"https://soco.college.harvard.edu/mobile_ws/v18/mobile_club_about?id={club_id}"
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status() # Raise error for bad status codes
        return response.json()
    except Exception as e:
        print(f"Error fetching ID {club_id}: {e}")
        return None

all_rows = []

for cid in club_ids:
    print(f"Fetching data for ID: {cid}...")
    data = get_club_data(cid)
    
    if data and 'club' in data:
        club = data['club']
        print(club)
        # Flattening complex fields (e.g., categories to a semicolon-separated string)
        categories = ""
        if club.get('categories') and isinstance(club.get('categories'), list):
            categories = "; ".join([cat.get('name', '') for cat in club.get('categories', []) if cat.get('name')])
        # 2. NEW: Flatten Officers
        # This creates a string like "President: Jack Botein; Treasurer: Nicholas Frumkin"
        officer_list = []
        for obs in club.get('officers', []):
            position = obs.get('position', 'Officer')
            student = obs.get('student', {})
            first_name = student.get('firstName', '')
            last_name = student.get('lastName', '')
            full_name = f"{first_name} {last_name}".strip()
            officer_list.append(f"{position}: {full_name}")
        
        officers_string = "; ".join(officer_list)

        # Creating a flat dictionary for the CSV row
        row = {
            "ID": club.get('id'),
            "Name": club.get('name'),
            "Type": club.get('groupTypeValue'),
            "Email": club.get('email'),
            "Website": club.get('websiteUrl'),
            "Instagram": club.get('instagram'),
            "Officers": officers_string,
            "Mission": (club.get('mission') or '').replace('\n', ' ').strip(),
            "What We Do": (club.get('whatWeDo') or '').replace('\n', ' ').strip(),
            "Categories": categories,
            "LinkedIn": club.get('linkedin')
        }
        all_rows.append(row)
if all_rows:
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=all_rows[0].keys())
        writer.writeheader()
        writer.writerows(all_rows)
    print(f"Successfully saved {len(all_rows)} clubs to {output_file}")

In [ ]:
import requests
import csv
import json

# 1. Configuration
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Mobile Safari/537.36",
    "Cookie": "fpestid=5YkdV-DmfjHeZUf4p6sb_1k8LRezxsxsC58LHpGtehPpZd7yPOEeTlLsbRyn_K2N-S2mRw; kndctr_1B6E34B85282A0AC0A490D44_AdobeOrg_identity=CiYxNTI3NzY2ODQxMjI3ODM0NDk0MjY2OTE0NTg1MDc2OTY5NzM2MFIQCIyPhrykMxgBKgNWQTYwAfABjI-GvKQz; AMCV_1B6E34B85282A0AC0A490D44%40AdobeOrg=MCMID|15277668412278344942669145850769697360; __gads=ID=a83a3424e082fd3a:T=1762136262:RT=1762136262:S=ALNI_MZNv532W2BLaGb5ap_t71wBBa5Pmw; _hjSessionUser_5278425=eyJpZCI6IjdjMDZlYjdhLWY0YzEtNTlmYS1hNzFhLTkzYmMyM2ZmZGM3OSIsImNyZWF0ZWQiOjE3NjMwMDg4MDgyNTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjSessionUser_1721990=eyJpZCI6ImI5NDk3ODkzLTY1MWYtNTEwMy05N2NmLTNhYzkwNmMwM2Q1OSIsImNyZWF0ZWQiOjE3NjMwNDc5MTcyNjcsImV4aXN0aW5nIjpmYWxzZX0=; _hjSessionUser_5132978=eyJpZCI6Ijk3OTE2MzE3LWNkMzMtNThiYi05OGM3LWRlMWVkZTNiZmFjOCIsImNyZWF0ZWQiOjE3NjMxNzUyNDk4NjYsImV4aXN0aW5nIjpmYWxzZX0=; _hjSessionUser_3372373=eyJpZCI6ImI3NDA5NjRlLTAwOTktNTBkNi05MmIyLWQwZDhjNTY3NDhmNSIsImNyZWF0ZWQiOjE3NjQ3MjQwODE5OTYsImV4aXN0aW5nIjp0cnVlfQ==; cf_clearance=JM2tGC6MrVlp1kYjfJTe9vuioU099R7AhmOL.3_Yzow-1764894820-1.2.1.1-fJRUGfurPRRuT7om4PD_tbHOksMJ_g9pH4jck64DrR7BJvuni_9Nj2PWES8ff1Dm3tkBdmCMysFZ0eQgeyf4mVulmB6AP960Ihm7PrSXdGNalOZ7pVLMSL0IENHUN9JbPY9Ge2WZIZnZ98GexvSe.IHZHEGsUaQg6tMeBgmufYWEKQc43gGdiXDxt4Y9c86fA8ndQ107KaxK8qIGM0961C6SOjI3hAv0WWxxyS_KC3A; visid_incap_881684=/g7SFpDSTK+WiJmcA4KG2X0yMmkAAAAAQUIPAAAAAACFkJKV8lsbu/IKeRLk6lOK; _ga_MN44P5KS85=GS2.1.s1764895975$o6$g1$t1764897472$j60$l0$h0; _ga_HRDZW9PB73=GS2.2.s1764894720$o1$g1$t1764897501$j60$l0$h0; _ga_REG2399202=GS2.1.s1764897711$o2$g0$t1764897711$j60$l0$h0; _ga_72NC9RC7VN=GS2.1.s1764894542$o2$g1$t1764897739$j33$l0$h629955202; visid_incap_2811896=rOT45HlGToelhjz49dx6RHBVMmkAAAAAQUIPAAAAAABihYsb7FN2f7wFKiSfLhsX; _ga_08YNBBX8C4=GS2.1.s1764906776$o2$g1$t1764907572$j60$l0$h0; _ga_831NSELVRG=GS2.1.s1765224826$o2$g0$t1765224855$j31$l0$h0; _ga_XDRYG9V1BN=GS2.1.s1765426404$o4$g1$t1765426465$j60$l0$h0; _ga_6S7711N1RR=GS2.1.s1765426586$o1$g0$t1765426650$j60$l0$h0; _ga_F8NW5WEMSK=GS2.1.s1765462528$o3$g0$t1765462550$j38$l0$h0; _ga_1MSJRLKMVP=GS2.1.s1765462528$o12$g0$t1765462550$j38$l0$h0; _ga_3185NP2KZP=GS2.1.s1765462528$o3$g0$t1765462550$j38$l0$h0; _ga_GMEVN96PYN=GS2.1.s1765462551$o3$g0$t1765462678$j60$l0$h0; _ga_J818J4JRZP=GS2.2.s1765463024$o2$g0$t1765463024$j60$l0$h0; _ga_SFXEYPKNXD=GS2.1.s1765463277$o7$g0$t1765463315$j22$l0$h0; _ga_YQT8F7VJ08=GS2.1.s1765463277$o7$g0$t1765463315$j22$l0$h0; _ga_WKZX29Q985=GS2.1.s1765506975$o10$g1$t1765508970$j60$l0$h0; _ga_0BL4ZQBM0W=GS2.1.s1765547176$o13$g1$t1765548042$j56$l0$h0; _ga_W2SR8DW607=GS2.1.s1765547195$o2$g0$t1765548042$j60$l0$h0; _ga_2PZD5XNYSL=GS2.1.s1765547195$o2$g0$t1765548042$j60$l0$h0; _ga_2MCEMGXSJF=GS2.1.s1765548287$o1$g0$t1765548318$j29$l0$h0; _ga_2ESJCEX86M=GS2.1.s1765547182$o1$g1$t1765548394$j60$l0$h0; nmstat=4f450c70-67cd-5076-44f5-775f278b6af2; _ga_KBGFFL5T7T=GS2.1.s1765570918$o4$g1$t1765572278$j60$l0$h0; _ga_XPLDGPNPHJ=GS2.1.s1765628270$o17$g1$t1765628595$j47$l0$h0; _ga_ZH44FE6MF7=GS2.1.s1765628270$o17$g1$t1765628595$j47$l0$h0; _ga_Y1ZL3JYDGL=GS2.1.s1765673795$o1$g0$t1765673798$j57$l0$h0; _ga_HY4R7TDDW6=GS2.1.s1765673795$o1$g0$t1765673798$j57$l0$h0; _ga_3CXC97RWEK=GS2.1.s1765765432$o2$g1$t1765766047$j60$l0$h0; _ga_8VM586D95L=GS2.1.s1765768720$o14$g1$t1765768752$j28$l0$h0; _ga_WF7T5MH29J=GS2.1.s1766006435$o1$g1$t1766007547$j60$l0$h0; CG.SessionID=42gjdopagzgcmb2zcgi2o1gs-603MqGQjbijRfBxjphSu3myEPe4%3d; cg_uid=2404742-+hq5SB7vZ8wL8QVOleqUz9vb8qp9nT93X62aqSHSStc=; _ga_QEYJY07BXE=GS2.1.s1766374804$o14$g1$t1766375893$j10$l0$h0; _gid=GA1.2.1732466459.1766377655; _clck=1m005al%5E2%5Eg22%5E0%5E2125; _clsk=1lbycy8%5E1766377874682%5E9%5E1%5Ed.clarity.ms%2Fcollect; _ga_ECX5BHYZT7=GS2.1.s1766377654$o9$g1$t1766377876$j57$l0$h0; cebs=1; _ce.clock_data=33%2C100.38.155.175%2C1%2Cf18b5213b6de2490ec9be218b0f025b0%2CChrome%2CUS; cebsp_=1; _ce.s=v~aee173ba801a53c35bd281eddbc9c24aec63d1e1~lcw~1766379328747~vir~returning~lva~1766379165807~vpv~6~v11ls~0ecccc80-def2-11f0-ac40-bfc9e8dc6631~vdva~1765151999999~v11.cs~462316~v11.s~0ecccc80-def2-11f0-ac40-bfc9e8dc6631~v11.vs~aee173ba801a53c35bd281eddbc9c24aec63d1e1~v11.fsvd~eyJ1cmwiOiJzZWFzLmhhcnZhcmQuZWR1L25ld3MvKi8qL2dyZWVuLXNwYWNlcy1yZWQtbGluZXMtY3JpbXNvbi1zY2llbnRpc3RzIiwicmVmIjoiaHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8iLCJ1dG0iOltdfQ%3D%3D~v11.sla~1766379166025~v11.wss~1766379166026~v11.ss~1766379166027~gtrk.la~mjgon97s~lcw~1766379329128; _ga_Q97GE6JKJ9=GS2.1.s1766379165$o16$g1$t1766379333$j60$l0$h0; _ga_FNNHYP076Y=GS2.1.s1766379165$o15$g1$t1766379333$j12$l0$h0; _ga=GA1.2.566895420.1765599137; _ga_ECPW5BRBQ7=GS2.2.s1766409177$o1$g1$t1766409177$j60$l0$h0",
    "Accept": "application/json"
}
output_file = 'harvard_officers_data.csv'
def get_officer_data(club_id):
    url = (
        f"https://soco.college.harvard.edu/mobile_ws/v17/mobile_group_page_officers?"
        f"range=0&limit=30&order=undefined&search_word=&param={club_id}&1766413421795"
    )
    
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        content = response.text.strip()
        if not content:
            return []
        try:
            return json.loads(content)
        except json.JSONDecodeError:
            decoder = json.JSONDecoder()
            data, index = decoder.raw_decode(content)
            return data
    except Exception as e:
        print(f"Error fetching officers for Club ID {club_id}: {e}")
        return []

all_officer_rows = []

for cid in club_ids:
    print(f"Fetching officers for Club ID: {cid}...")
    officers = get_officer_data(cid)
    for member in officers:
        row = {
            "ClubID": cid,
            "FirstName": member.get("p0"),
            "LastName": member.get("p1"),
            "StudentType": member.get("p2"),
            "Position": member.get("p7"),
            "Email": member.get("p8"),
            "FullName": member.get("p13", "").replace('\n', ' ').strip()
        }
        all_officer_rows.append(row)
if all_officer_rows:
    headers = ["ClubID", "FirstName", "LastName", "StudentType", "Position", "Email", "FullName"]
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        writer.writerows(all_officer_rows)
    print(f"Successfully saved {len(all_officer_rows)} officers to {output_file}")
else:
    print("No officer data collected.")

Fetching officers for Club ID: 36151...
Successfully saved 3 officers to harvard_officers_data.csv
